In [1]:
%load_ext autoreload
%autoreload 2


You will need to first create a kaggle account, the download your kaggle API key.
Info on how to get your api key (kaggle.json) here:
https://github.com/Kaggle/kaggle-api#api-credentials

Place the kaggle.json file in the correct directory (~/.kaggle).

If you are running on google colab, then in the next cell you'll be presented
with a widget. Use it to browse to the kaggle.json file.
(Note: You might have to show hidden files/folders in the file browser) 


In [2]:
try:
    import google.colab
    from google.colab import files
    files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
    !pip install -q kaggle
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    IN_COLAB = True
except:
    IN_COLAB = False


Now that your system is configured to use kaggle, whether local or running on colab,
we check for the existence of the datadir, and if it's not there we download the data
as a zip file, then unzip it, then delete the zip file.
This dataset may still be private, in which case you will have to ask Chad to grant you
access.


In [3]:
from pathlib import Path
datadir = Path.cwd() / 'data' / 'kjm_ecog'
if not (datadir / 'converted').is_dir():
    !kaggle datasets download -d cboulay/kjm-ecog-faces-basic
    data_path = Path('kjm-ecog-faces-basic.zip')
    import zipfile
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        zip_ref.extractall(datadir)
    data_path.unlink()


We can now start working with the data.
We will import a single file that contains processed band-power data.


In [4]:
from data.utils.fileio import from_neuropype_h5
test_file = datadir / 'converted' / 'faces_basic' / 'de_bp.h5'
chunks = from_neuropype_h5(test_file)
    

In [6]:
import pandas as pd
for chnk in chunks:
    ax_types = [_['type'] for _ in chnk[1]['axes']]
    print(chnk[0], chnk[1]['data'].shape, ax_types)
    
    instance_axis = chnk[1]['axes'][ax_types.index('instance')]
    print(pd.unique(instance_axis['data']['Marker']))


signals (603, 17, 31, 7) ['instance', 'time', 'space', 'frequency']
['ISI' 'house' 'face']
